# 以Transformers套件實作問答(Question Answering)功能

In [5]:
# 載入相關套件
from transformers import pipeline

In [6]:
# 載入模型
nlp = pipeline("question-answering", 'distilbert-base-cased-distilled-squad')

All PyTorch model weights were used when initializing TFDistilBertForQuestionAnswering.

All the weights of TFDistilBertForQuestionAnswering were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForQuestionAnswering for predictions without further training.


In [7]:
# 訓練資料
context = r"Extractive Question Answering is the task of extracting an answer " + \
"from a text given a question. An example of a question answering " + \
"dataset is the SQuAD dataset, which is entirely based on that task. " + \
"If you would like to fine-tune a model on a SQuAD task, you may " + \
"leverage the examples/question-answering/run_squad.py script."

In [8]:
# 測試 2 筆
result = nlp(question="What is extractive question answering?", context=context)
print(f"Answer: '{result['answer']}', score: {round(result['score'], 4)}",
        f", start: {result['start']}, end: {result['end']}")

print()

result = nlp(question="What is a good example of a question answering dataset?", 
             context=context)
print(f"Answer: '{result['answer']}', score: {round(result['score'], 4)}",
        f", start: {result['start']}, end: {result['end']}")

Answer: 'the task of extracting an answer from a text given a question', score: 0.6226 , start: 33, end: 94

Answer: 'SQuAD dataset', score: 0.5053 , start: 146, end: 159


## 結合Tokenizer

In [9]:
from transformers import AutoTokenizer, TFAutoModelForQuestionAnswering
import tensorflow as tf

# 結合分詞器(Tokenizer)
tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")
model = TFAutoModelForQuestionAnswering.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")

All PyTorch model weights were used when initializing TFBertForQuestionAnswering.

All the weights of TFBertForQuestionAnswering were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForQuestionAnswering for predictions without further training.


In [10]:
# 訓練資料
text = r"""
🤗 Transformers (formerly known as pytorch-transformers and pytorch-pretrained-bert) provides general-purpose
architectures (BERT, GPT-2, RoBERTa, XLM, DistilBert, XLNet…) for Natural Language Understanding (NLU) and Natural
Language Generation (NLG) with over 32+ pretrained models in 100+ languages and deep interoperability between
TensorFlow 2.0 and PyTorch.
"""

In [11]:
# 問題
questions = [
    "How many pretrained models are available in 🤗 Transformers?",
    "What does 🤗 Transformers provide?",
    "🤗 Transformers provides interoperability between which frameworks?",
]

In [12]:
# 推測答案
for question in questions:
    inputs = tokenizer(question, text, add_special_tokens=True, return_tensors="tf")
    input_ids = inputs["input_ids"].numpy()[0]
    outputs = model(inputs)
    
    answer_start_scores = outputs.start_logits
    answer_end_scores = outputs.end_logits
    answer_start = tf.argmax(answer_start_scores, axis=1).numpy()[0]  
    answer_end = (tf.argmax(answer_end_scores, axis=1) + 1).numpy()[0]  
    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens
                                            (input_ids[answer_start:answer_end]))
    
    print(f"Question: {question}")
    print(f"Answer: {answer}\n")

Question: How many pretrained models are available in 🤗 Transformers?
Answer: over 32 +

Question: What does 🤗 Transformers provide?
Answer: general - purpose architectures

Question: 🤗 Transformers provides interoperability between which frameworks?
Answer: tensorflow 2. 0 and pytorch

